In [ ]:
''' single eye random image selection approx 500 each eye'''
##### import os
import shutil
import random
from collections import defaultdict, Counter

# ---------------------------------------------------------
# 1. BALANCED IMAGE SELECTION (UNIQUE PER BENEFICIARY)
# ---------------------------------------------------------
def select_balanced_images(source_dirs, output_dir, used_beneficiaries, target_per_eye, seed=72):
    """
    Selects balanced images across 6 eye views while ensuring:
    ✅ Each beneficiary is used only once globally (across all Hb thresholds)
    ✅ Each eye view has ~target_per_eye images
    """
    random.seed(seed)
    os.makedirs(output_dir, exist_ok=True)

    suffix_map = {
        'left1': '_left_eye_1.png',
        'left2': '_left_eye_2.png',
        'left3': '_left_eye_3.png',
        'right1': '_right_eye_1.png',
        'right2': '_right_eye_2.png',
        'right3': '_right_eye_3.png',
    }

    # Collect images per beneficiary
    beneficiary_images = defaultdict(dict)
    for key, src_dir in source_dirs.items():
        suffix = suffix_map[key]
        if not os.path.exists(src_dir):
            continue
        for fname in os.listdir(src_dir):
            if fname.endswith(suffix):
                b_id = fname.replace(suffix, "")
                if b_id in used_beneficiaries:
                    continue
                beneficiary_images[b_id][key] = os.path.join(src_dir, fname)

    beneficiaries = list(beneficiary_images.keys())
    random.shuffle(beneficiaries)

    counts = {k: 0 for k in suffix_map.keys()}

    # Pick balanced images
    for b_id in beneficiaries:
        for eye_key in sorted(counts, key=counts.get):  # always choose least-filled eye first
            if eye_key in beneficiary_images[b_id] and counts[eye_key] < target_per_eye:
                src = beneficiary_images[b_id][eye_key]
                shutil.copy(src, os.path.join(output_dir, os.path.basename(src)))
                counts[eye_key] += 1
                used_beneficiaries.add(b_id)
                break

        # Stop early if all targets are filled
        if all(c >= target_per_eye for c in counts.values()):
            break

    print(f"✅ Copied {sum(counts.values())} images → {output_dir}")
    print(f"📊 Per-eye distribution: {counts}")
    return counts

# ---------------------------------------------------------
# 2. ENSURE DIRECTORY EXISTS
# ---------------------------------------------------------
def ensure_directory(path):
    os.makedirs(path, exist_ok=True)

# ---------------------------------------------------------
# 3. TRIM DATASET TO EXACTLY 2990 IMAGES
# ---------------------------------------------------------
def trim_to_target_images(folders, target_total=2990, seed=42):
    random.seed(seed)
    folder_images = {
        f: [os.path.join(f, x) for x in os.listdir(f) if x.endswith(".png")]
        for f in folders
    }
    total_images = sum(len(imgs) for imgs in folder_images.values())

    print(f"📂 Current total images: {total_images}")
    if total_images <= target_total:
        print(f"✅ No trimming needed, already ≤ {target_total}.")
        return

    excess = total_images - target_total
    print(f"⚠️ Need to delete {excess} images to maintain {target_total} total.")

    per_folder_delete = excess // len(folders)
    remaining = excess % len(folders)
    deleted_count = 0

    for i, (folder, imgs) in enumerate(folder_images.items()):
        random.shuffle(imgs)
        num_to_delete = per_folder_delete + (1 if i < remaining else 0)
        for img in imgs[:num_to_delete]:
            os.remove(img)
            deleted_count += 1
        print(f"🗑️ Deleted {num_to_delete} images from: {folder}")

    print(f"✅ Done! Deleted {deleted_count} images total.")
    print(f"🎯 Final dataset size: {target_total}")

# ---------------------------------------------------------
# 4. VERIFY FINAL DATASET DISTRIBUTION
# ---------------------------------------------------------
def verify_image_distribution(base_path, folders):
    patterns = [
        "_left_eye_1.png", "_left_eye_2.png", "_left_eye_3.png",
        "_right_eye_1.png", "_right_eye_2.png", "_right_eye_3.png"
    ]
    image_counts = Counter()
    beneficiary_ids = set()

    for folder in folders:
        folder_path = os.path.join(base_path, folder)
        for fname in os.listdir(folder_path):
            for pattern in patterns:
                if fname.endswith(pattern):
                    image_counts[pattern] += 1
            b_id = fname.split("_left_eye_")[0] if "_left_eye_" in fname else fname.split("_right_eye_")[0]
            beneficiary_ids.add(b_id)

    print("\n📊 Final Image Counts by Eye Type:")
    for pattern in patterns:
        print(f"{pattern}: {image_counts[pattern]}")

    left_total = sum(image_counts[p] for p in patterns if "left_eye" in p)
    right_total = sum(image_counts[p] for p in patterns if "right_eye" in p)
    grand_total = left_total + right_total

    print("\n🔹 Final Totals:")
    print(f"  Total LEFT eye images : {left_total}")
    print(f"  Total RIGHT eye images: {right_total}")
    print(f"  Grand Total           : {grand_total}")
    print(f"  Unique Beneficiaries  : {len(beneficiary_ids)}")

# ---------------------------------------------------------
# 5. MAIN EXECUTION (AUTOMATED FOR MULTIPLE Hb THRESHOLDS)
# ---------------------------------------------------------
'''
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
hb_thresholds = [    "9_0"] #and other HB thresholds
target_images_total = 3000
target_per_eye = 495  # ~490–498 range

used_beneficiaries = set()

for hb in hb_thresholds:
    print(f"\n🚀 Processing Hb threshold: {hb.replace('_', '.')} g/dL")

    output_dir = os.path.join(base_path, f"hb_{hb}")
    ensure_directory(output_dir)

    output_base = os.path.join(output_dir, "conjunctiva_extracted")
    ensure_directory(output_base)

    # Output folders for this Hb threshold
    output_paths = [
        os.path.join(output_base, 'anemic_train_roi'),
        os.path.join(output_base, 'anemic_not_train_roi'),
        os.path.join(output_base, 'anemic_val_roi'),
        os.path.join(output_base, 'anemic_not_val_roi'),
        os.path.join(output_base, 'anemic_test_roi'),
        os.path.join(output_base, 'anemic_not_test_roi')
    ]
    for path in output_paths:
        ensure_directory(path)

    def make_full_path(subdirs):
        return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

    # Define mappings dynamically for each Hb threshold
    def dirs_for(split_type, anemic=True):
        prefix = "anemic" if anemic else "anemic_not"
        return make_full_path({
            'left1': f"tri_left_eye/left_eye_1_hb_less_than_{hb}/conjunctiva_extracted/{prefix}_{split_type}_roi/",
            'left2': f"tri_left_eye/left_eye_2_hb_less_than_{hb}/conjunctiva_extracted/{prefix}_{split_type}_roi/",
            'left3': f"tri_left_eye/left_eye_3_hb_less_than_{hb}/conjunctiva_extracted/{prefix}_{split_type}_roi/",
            'right1': f"tri_right_eye/right_eye_1_hb_less_than_{hb}/conjunctiva_extracted/{prefix}_{split_type}_roi/",
            'right2': f"tri_right_eye/right_eye_2_hb_less_than_{hb}/conjunctiva_extracted/{prefix}_{split_type}_roi/",
            'right3': f"tri_right_eye/right_eye_3_hb_less_than_{hb}/conjunctiva_extracted/{prefix}_{split_type}_roi/"
        })

    # Run balanced selection for all 6 splits per Hb threshold
    select_balanced_images(dirs_for("train"), os.path.join(output_base, 'anemic_train_roi'), used_beneficiaries, target_per_eye)
    select_balanced_images(dirs_for("train", anemic=False), os.path.join(output_base, 'anemic_not_train_roi'), used_beneficiaries, target_per_eye)
    select_balanced_images(dirs_for("val"), os.path.join(output_base, 'anemic_val_roi'), used_beneficiaries, target_per_eye)
    select_balanced_images(dirs_for("val", anemic=False), os.path.join(output_base, 'anemic_not_val_roi'), used_beneficiaries, target_per_eye)
    select_balanced_images(dirs_for("test"), os.path.join(output_base, 'anemic_test_roi'), used_beneficiaries, target_per_eye)
    select_balanced_images(dirs_for("test", anemic=False), os.path.join(output_base, 'anemic_not_test_roi'), used_beneficiaries, target_per_eye)

    # Trim dataset to exactly 2990 images for this Hb threshold
    trim_to_target_images(output_paths, target_total=target_images_total)

    # Verify distribution per Hb threshold
    verify_image_distribution(output_base, [
        "anemic_train_roi", "anemic_not_train_roi",
        "anemic_val_roi", "anemic_not_val_roi",
        "anemic_test_roi", "anemic_not_test_roi"
    ])
'''


'\nbase_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"\nhb_thresholds = [    "9_0"] #and other HB thresholds\ntarget_images_total = 3000\ntarget_per_eye = 495  # ~490–498 range\n\nused_beneficiaries = set()\n\nfor hb in hb_thresholds:\n    print(f"\n🚀 Processing Hb threshold: {hb.replace(\'_\', \'.\')} g/dL")\n\n    output_dir = os.path.join(base_path, f"hb_{hb}")\n    ensure_directory(output_dir)\n\n    output_base = os.path.join(output_dir, "conjunctiva_extracted")\n    ensure_directory(output_base)\n\n    # Output folders for this Hb threshold\n    output_paths = [\n        os.path.join(output_base, \'anemic_train_roi\'),\n        os.path.join(output_base, \'anemic_not_train_roi\'),\n        os.path.join(output_base, \'anemic_val_roi\'),\n        os.path.join(output_base, \'anemic_not_val_roi\'),\n        os.path.join(output_base, \'anemic_test_roi\'),\n        os.path.join(output_base, \'anemic_not_test_roi\')\n    ]\n    for path in outp

In [ ]:
import os
from collections import Counter

# Base path for images
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hb_9_0/conjunctiva_extracted"

# List of all 6 folders to scan
folders = [
    "anemic_train_roi",
    "anemic_not_train_roi",
    "anemic_val_roi",
    "anemic_not_val_roi",
    "anemic_test_roi",
    "anemic_not_test_roi"
]

# Patterns to track
patterns = [
    "_left_eye_1.png",
    "_left_eye_2.png",
    "_left_eye_3.png",
    "_right_eye_1.png",
    "_right_eye_2.png",
    "_right_eye_3.png"
]

# Counter for image types
image_counts = Counter()

# Loop through folders and count images by pattern
for folder in folders:
    folder_path = os.path.join(base_path, folder)
    for fname in os.listdir(folder_path):
        for pattern in patterns:
            if fname.endswith(pattern):
                image_counts[pattern] += 1

# Display results
print("📊 Image counts by type:\n")
for pattern in patterns:
    print(f"{pattern}: {image_counts[pattern]}")

print("\n🔹 Totals:")
left_total = sum(image_counts[p] for p in patterns if "left_eye" in p)
right_total = sum(image_counts[p] for p in patterns if "right_eye" in p)
grand_total = left_total + right_total

print(f"  Total LEFT eye images : {left_total}")
print(f"  Total RIGHT eye images: {right_total}")
print(f"  Grand Total          : {grand_total}")


📊 Image counts by type:

_left_eye_1.png: 502
_left_eye_2.png: 501
_left_eye_3.png: 500
_right_eye_1.png: 500
_right_eye_2.png: 499
_right_eye_3.png: 498

🔹 Totals:
  Total LEFT eye images : 1503
  Total RIGHT eye images: 1497
  Grand Total          : 3000


In [ ]:
import os
from collections import Counter

# Base path for images
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/hb_9_0/conjunctiva_extracted"

# Folders to check
folders = [
    "anemic_train_roi",
    "anemic_not_train_roi",
    "anemic_val_roi",
    "anemic_not_val_roi",
    "anemic_test_roi",
    "anemic_not_test_roi"
]

# Collect all base IDs (filenames without eye suffixes)
base_ids = []

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    for fname in os.listdir(folder_path):
        if fname.endswith(".png"):
            # Remove suffix: everything after first "_left_eye" or "_right_eye"
            if "_left_eye" in fname:
                base_id = fname.split("_left_eye")[0]
            elif "_right_eye" in fname:
                base_id = fname.split("_right_eye")[0]
            else:
                base_id = os.path.splitext(fname)[0]  # fallback

            base_ids.append(base_id)

# Count total vs unique
total_files = len(base_ids)
unique_ids = len(set(base_ids))

print(f"📂 Total images          : {total_files}")
print(f"🔹 Unique beneficiary IDs: {unique_ids}")

if total_files == unique_ids:
    print("✅ All images are unique!")
else:
    print(f"⚠️ Found {total_files - unique_ids} duplicate images.")

    # Find duplicates explicitly
    duplicates = [item for item, count in Counter(base_ids).items() if count > 1]
    print(f"🔁 Duplicate IDs ({len(duplicates)}):")
    for dup in duplicates:
        print(f"  - {dup}")


📂 Total images          : 3000
🔹 Unique beneficiary IDs: 3000
✅ All images are unique!


In [ ]:
import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

# =========================
# GLOBAL CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True

N_SPLITS = 5
RESOLUTIONS = [224]
EPOCHS_CV = 75
BATCH_CV = 18
LR_CV = 0.00018

# =========================
# DEVICE + DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "hexa-eye_hb_90_repro_bestfold_only_with_age_days")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

# Define directories
train_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS: base id handling and hexa intersections
# =========================
def base_from(fname, suffix):
    if not fname.endswith(suffix):
        return None
    return fname[: -len(suffix)]

def common_bases_across_six(dirs_map):
    suffixes = {
        'left1': '_left_eye_1.png',
        'left2': '_left_eye_2.png',
        'left3': '_left_eye_3.png',
        'right1': '_right_eye_1.png',
        'right2': '_right_eye_2.png',
        'right3': '_right_eye_3.png',
    }
    bases_sets = []
    for k in ['left1','left2','left3','right1','right2','right3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            print(f"⚠️ Folder missing: {folder}")
            return []
        names = sorted([f for f in os.listdir(folder) if f.endswith(suffixes[k])])
        bases = set()
        for f in names:
            b = base_from(f, suffixes[k])
            if b:
                bases.add(b)
        bases_sets.append(bases)
    common = set.intersection(*bases_sets) if bases_sets else set()
    return sorted(list(common))

def load_hexa_images_by_bases(dirs_map, bases):
    out = {'l1': [], 'l2': [], 'l3': [], 'r1': [], 'r2': [], 'r3': []}
    key_map = {
        'l1': ('left1',  '_left_eye_1.png'),
        'l2': ('left2',  '_left_eye_2.png'),
        'l3': ('left3',  '_left_eye_3.png'),
        'r1': ('right1', '_right_eye_1.png'),
        'r2': ('right2', '_right_eye_2.png'),
        'r3': ('right3', '_right_eye_3.png'),
    }
    ok = 0
    for b in bases:
        imgs = {}
        failed = False
        for short_k, (long_k, suf) in key_map.items():
            path = os.path.join(dirs_map[long_k], b + suf)
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[short_k] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if failed:
            continue
        for k in out.keys():
            out[k].append(imgs[k])
        ok += 1
    return out, ok

def prepare_dataset_strict(anemic_dirs, non_dirs, split_name="(split)"):
    bases_anemic = common_bases_across_six(anemic_dirs)
    bases_non = common_bases_across_six(non_dirs)
    print(f"\n🔎 {split_name} - discovered bases: anemic={len(bases_anemic)}, non-anemic={len(bases_non)}")

    imgs_a, _ = load_hexa_images_by_bases(anemic_dirs, bases_anemic)
    imgs_n, _ = load_hexa_images_by_bases(non_dirs,  bases_non)

    data = {
        'l1': imgs_a['l1'] + imgs_n['l1'],
        'l2': imgs_a['l2'] + imgs_n['l2'],
        'l3': imgs_a['l3'] + imgs_n['l3'],
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'label': [1]*len(imgs_a['l1']) + [0]*len(imgs_n['l1']),
        'base_ids': bases_anemic + bases_non  # ✅ Store base IDs
    }

    print(f"✅ {split_name}: hexa-matched samples -> anemic={len(imgs_a['l1'])}, non-anemic={len(imgs_n['l1'])}, total={len(data['label'])}")
    return data

# =========================
# LOAD DATAFRAME WITH AGE, DAYS, AND LIGHTING
# =========================
df = pd.read_csv("/home/ubuntu/anemia-storage/hb_mobilenet/dataset_imputed.csv")

# ✅ Fix: Extract base_id from filename like 'cr4n1a07ch0lnecm8a3g_right_eye_1.png' → 'cr4n1a07ch0lnecm8a3g'
def extract_base_id(filename):
    for suffix in ['_right_eye_1.png', '_right_eye_2.png', '_right_eye_3.png',
                   '_left_eye_1.png', '_left_eye_2.png', '_left_eye_3.png']:
        if filename.endswith(suffix):
            return filename[:-len(suffix)]
    return filename

# ✅ Create feature_map using clean base_id
feature_map = {}
for _, row in df.iterrows():
    raw_id = row['right_eye_1']  # Use any image column
    base_id = extract_base_id(raw_id)
    age = float(row['age_at_registration_final'])
    days = float(row['days_since_lmp_final'])
    rl1 = float(row['right_eye_1_light'])
    ll1 = float(row['left_eye_1_light'])
    rl2 = float(row['right_eye_2_light'])
    ll2 = float(row['left_eye_2_light'])
    rl3 = float(row['right_eye_3_light'])
    ll3 = float(row['left_eye_3_light'])

    feature_map[base_id] = [age, days, rl1, ll1, rl2, ll2, rl3, ll3]

# =========================
# DATASET / DATALOADER
# =========================
class HexaEyeDatasetWithFeatures(Dataset):
    def __init__(self, data, transform, feature_map):
        self.data = data
        self.transform = transform
        self.feature_map = feature_map

    def __len__(self):
        return len(self.data['label'])

    def __getitem__(self, idx):
        images = [self.data[k][idx] for k in ['l1','l2','l3','r1','r2','r3']]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        base_id = self.data['base_ids'][idx]
        features = self.feature_map.get(base_id, [0.0]*8)
        return images, label, torch.tensor(features, dtype=torch.float32)

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type == 'cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
        persistent_workers=False
    )

# =========================
# MODEL (6x ResNet18 + MLP head + Feature Fusion)
# =========================
class HexaResNetWithFeatures(nn.Module):
    def __init__(self):
        super().__init__()
        def res():
            m = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
            m.fc = nn.Identity()
            return m
        self.models = nn.ModuleList([res() for _ in range(6)])
        self.fc = nn.Sequential(
            nn.Linear(6*512, 1024),
            nn.LayerNorm(1024),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128)
        )
        self.feature_fc = nn.Sequential(
            nn.Linear(8, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        self.final_fc = nn.Sequential(
            nn.Linear(128 + 32, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, *x, features=None):
        feats = [model(xi) for model, xi in zip(self.models, x)]
        x_img = torch.cat(feats, dim=1)
        x_img = self.fc(x_img)
        x_feat = self.feature_fc(features)
        x = torch.cat([x_img, x_feat], dim=1)
        return self.final_fc(x)

# =========================
# EVALUATION
# =========================
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    for imgs, labels, features in loader:
        imgs = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        features = features.to(device).float()
        out = model(*imgs, features=features)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(all_labels)) < 2:
        p = r = f1 = auc = float('nan')
        acc = accuracy_score(all_labels, all_preds)
        tn = fp = fn = tp = 0
        try:
            tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
        except Exception:
            pass
        return p, r, f1, acc, auc, tp, tn, fp, fn

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn

# =========================
# PREPARE DATASETS
# =========================
train_data = prepare_dataset_strict(train_dirs_anemic, train_dirs_non, "TRAIN")
val_data = prepare_dataset_strict(val_dirs_anemic, val_dirs_non, "VAL")
test_data = prepare_dataset_strict(test_dirs_anemic, test_dirs_non, "TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No hexa-matched TRAIN samples found. Check paths/filenames.")
if len(val_data['label']) == 0:
    print("⚠️ No hexa-matched VAL samples found.")
if len(test_data['label']) == 0:
    print("⚠️ No hexa-matched TEST samples found.")

# =========================
# 5-FOLD CV
# =========================
results = []

for resolution in RESOLUTIONS:
    print(f"\n===== Processing resolution: {resolution} =====")
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    if len(train_data['label']) < N_SPLITS:
        raise RuntimeError(f"Too few train samples for {N_SPLITS}-fold CV.")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(train_data['label'])
    fold = 1
    cv_index_records = []

    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- Fold {fold} ---")

        cv_index_records.append({
            "fold": fold,
            "train_indices": train_idx.tolist(),
            "val_indices": val_idx.tolist()
        })

        train_subset = {k: [v[i] for i in train_idx] for k, v in train_data.items()}
        val_subset = {k: [v[i] for i in val_idx] for k, v in train_data.items()}

        train_loader = make_loader(HexaEyeDatasetWithFeatures(train_subset, train_tf, feature_map), batch_size=BATCH_CV, shuffle=True)
        val_loader = make_loader(HexaEyeDatasetWithFeatures(val_subset, eval_tf, feature_map), batch_size=BATCH_CV, shuffle=False)

        model = HexaResNetWithFeatures().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels, features in train_loader:
                imgs = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                features = features.to(device).float()
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(*imgs, features=features)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()
            print(f"Epoch [{epoch+1}/{EPOCHS_CV}] 🔁 Loss: {total_loss:.6f}")

        val_metrics = evaluate(model, val_loader)
        row = {
            'Resolution': resolution,
            'Fold': fold,
            'Epochs': EPOCHS_CV,
            'BatchSize': BATCH_CV,
            'LearningRate': LR_CV,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8]
        }
        results.append(row)
        print(row)

        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"cv_fold_{fold}_res{resolution}.pt")
            torch.save({
                'model_state': model.state_dict(),
                'seed': SEED,
                'resolution': resolution
            }, fold_path)

        fold += 1

    # Save results
    if results:
        pd.DataFrame(results).to_csv(os.path.join(output_dir, f"{resolution}_val_cross_validation_results_hexa.csv"), index=False)
    pd.DataFrame(cv_index_records).to_json(os.path.join(output_dir, f"{resolution}_cv_indices.json"), orient='records', indent=2)

    # Select best fold
    results_df = pd.DataFrame(results)
    results_df['min_PR'] = results_df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = results_df[(results_df['Val_Precision'] >= 0.90) & (results_df['Val_Recall'] >= 0.90)]
    best = candidates.sort_values(['Val_F1','Val_AUC','min_PR'], ascending=False).iloc[0] if len(candidates) > 0 else \
            results_df.sort_values(['min_PR','Val_F1','Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # ✅ Load best fold model for test evaluation
    best_model = HexaResNetWithFeatures().to(device)
    best_model_path = os.path.join(output_dir, f"cv_fold_{best_fold}_res{resolution}.pt")
    state = torch.load(best_model_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Evaluate on TEST using best fold
    if len(test_data['label']) > 0:
        test_loader = make_loader(HexaEyeDatasetWithFeatures(test_data, eval_tf, feature_map), batch_size=BATCH_CV, shuffle=False)
        test_metrics = evaluate(best_model, test_loader)
        print("\n📊 TEST (best-fold model) Results")
        print(f"Precision: {test_metrics[0]:.4f}, Recall: {test_metrics[1]:.4f}, F1: {test_metrics[2]:.4f}")
        print(f"Accuracy:  {test_metrics[3]:.4f}, AUC: {test_metrics[4]:.4f}")

        pd.DataFrame([{
            'ChosenFold': best_fold,
            'Test_Precision': test_metrics[0], 'Test_Recall': test_metrics[1], 'Test_F1': test_metrics[2],
            'Test_Accuracy': test_metrics[3], 'Test_AUC': test_metrics[4], 'Test_TP': test_metrics[5],
            'Test_TN': test_metrics[6], 'Test_FP': test_metrics[7], 'Test_FN': test_metrics[8]
        }]).to_csv(os.path.join(output_dir, f"{resolution}_bestfold_test_results_hexa.csv"), index=False)

print("\n✅ Hexa-eye: Done. 5-fold CV, best-fold selection, and TEST evaluation are fully reproducible.")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

🔎 TRAIN - discovered bases: anemic=235, non-anemic=1824
✅ TRAIN: hexa-matched samples -> anemic=235, non-anemic=1824, total=2059

🔎 VAL - discovered bases: anemic=37, non-anemic=291
✅ VAL: hexa-matched samples -> anemic=37, non-anemic=291, total=328

🔎 TEST - discovered bases: anemic=28, non-anemic=237
✅ TEST: hexa-matched samples -> anemic=28, non-anemic=237, total=265

===== Processing resolution: 224 =====

--- Fold 1 ---
Epoch [1/75] 🔁 Loss: 34.002423
Epoch [2/75] 🔁 Loss: 33.531065
Epoch [3/75] 🔁 Loss: 32.803141
Epoch [4/75] 🔁 Loss: 33.063180
Epoch [5/75] 🔁 Loss: 31.811949
Epoch [6/75] 🔁 Loss: 31.779203
Epoch [7/75] 🔁 Loss: 30.522110
Epoch [8/75] 🔁 Loss: 30.739449
Epoch [9/75] 🔁 Loss: 30.665748
Epoch [10/75] 🔁 Loss: 29.967029
Epoch [11/75] 🔁 Loss: 29.971080
Epoch [12/75] 🔁 Loss: 29.074833
Epoch [13/75] 🔁 Loss: 28.495415
Epoch [14/75] 🔁 Loss: 28.998241
Epoch [15/75] 🔁 Loss: 27.142231
Epoch [16/75] 🔁 Loss: 26.677579
Epoch [17/75] 🔁 Loss:

In [ ]:
# 🔍 Check if additional features were loaded and used

# Step 1: Verify feature_map is created
print("✅ Feature Map Created:", 'feature_map' in locals())
if 'feature_map' in locals():
    print(f"  ➤ Number of base IDs in feature_map: {len(feature_map)}")
    # Sample one entry
    sample_base = list(feature_map.keys())[0]
    sample_features = feature_map[sample_base]
    print(f"  ➤ Sample features for '{sample_base}': {sample_features}")
    print(f"  ➤ Number of features: {len(sample_features)}")

# Step 2: Verify model structure accepts 8 features
model = HexaResNetWithFeatures()
print("\n✅ Model Structure:")
print("  ➤ Input features size:", model.feature_fc[0].in_features)  # Should be 8
print("  ➤ Output features size:", model.feature_fc[-1].out_features)  # Should be 32

# After fixing feature_map
sample_base = list(feature_map.keys())[0]
print("Sample base_id:", sample_base)
print("Available in train base_ids?", sample_base in train_data['base_ids'])
print("Features:", feature_map[sample_base])

✅ Feature Map Created: True
  ➤ Number of base IDs in feature_map: 3025
  ➤ Sample features for 'cr4n1a07ch0lnecm8a3g': [28.0, 240.0, 317.99252, 337.02753, 382.99503, 407.97, 382.99503, 330.0075]
  ➤ Number of features: 8

✅ Model Structure:
  ➤ Input features size: 8
  ➤ Output features size: 32
Sample base_id: cr4n1a07ch0lnecm8a3g
Available in train base_ids? True
Features: [28.0, 240.0, 317.99252, 337.02753, 382.99503, 407.97, 382.99503, 330.0075]


In [ ]:
import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

# =========================
# GLOBAL CONFIG
# =========================
SEED = 42
NUM_WORKERS = 0
PIN_MEMORY = False
USE_AMP = True
SAVE_EVERY_FOLD_MODEL = True

N_SPLITS = 5
RESOLUTIONS = [224]
EPOCHS_CV =120
BATCH_CV = 24
LR_CV = 0.00022

# =========================
# DEVICE + DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "tri-right-eye_hb_90_repro_bestfold_only_with_age_days")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

# Define directories (RIGHT EYE ONLY)
train_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'right1': 'tri_right_eye/right_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right2': 'tri_right_eye/right_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'right3': 'tri_right_eye/right_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS: base id handling and tri-eye intersections
# =========================
def base_from(fname, suffix):
    if not fname.endswith(suffix):
        return None
    return fname[: -len(suffix)]

def common_bases_across_three(dirs_map):
    suffixes = {
        'right1': '_right_eye_1.png',
        'right2': '_right_eye_2.png',
        'right3': '_right_eye_3.png',
    }
    bases_sets = []
    for k in ['right1','right2','right3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            print(f"⚠️ Folder missing: {folder}")
            return []
        names = sorted([f for f in os.listdir(folder) if f.endswith(suffixes[k])])
        bases = set()
        for f in names:
            b = base_from(f, suffixes[k])
            if b:
                bases.add(b)
        bases_sets.append(bases)
    common = set.intersection(*bases_sets) if bases_sets else set()
    return sorted(list(common))

def load_tri_images_by_bases(dirs_map, bases):
    out = {'r1': [], 'r2': [], 'r3': []}
    key_map = {
        'r1': ('right1', '_right_eye_1.png'),
        'r2': ('right2', '_right_eye_2.png'),
        'r3': ('right3', '_right_eye_3.png'),
    }
    ok = 0
    for b in bases:
        imgs = {}
        failed = False
        for short_k, (long_k, suf) in key_map.items():
            path = os.path.join(dirs_map[long_k], b + suf)
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[short_k] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if failed:
            continue
        for k in out.keys():
            out[k].append(imgs[k])
        ok += 1
    return out, ok

def prepare_dataset_strict(anemic_dirs, non_dirs, split_name="(split)"):
    bases_anemic = common_bases_across_three(anemic_dirs)
    bases_non = common_bases_across_three(non_dirs)
    print(f"\n🔎 {split_name} - discovered bases: anemic={len(bases_anemic)}, non-anemic={len(bases_non)}")

    imgs_a, _ = load_tri_images_by_bases(anemic_dirs, bases_anemic)
    imgs_n, _ = load_tri_images_by_bases(non_dirs, bases_non)

    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1']),
        'base_ids': bases_anemic + bases_non  # ✅ Store base IDs
    }

    print(f"✅ {split_name}: tri-matched samples -> anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    return data

# =========================
# LOAD DATAFRAME WITH AGE, DAYS, AND LIGHTING
# =========================
df = pd.read_csv("/home/ubuntu/anemia-storage/hb_mobilenet/dataset_imputed.csv")

# ✅ Extract base_id from filename
def extract_base_id(filename):
    for suffix in ['_right_eye_1.png', '_right_eye_2.png', '_right_eye_3.png']:
        if filename.endswith(suffix):
            return filename[:-len(suffix)]
    return filename

# ✅ Create feature_map using clean base_id
feature_map = {}
for _, row in df.iterrows():
    raw_id = row['right_eye_1']
    base_id = extract_base_id(raw_id)
    age = float(row['age_at_registration_final'])
    days = float(row['days_since_lmp_final'])
    rl1 = float(row['right_eye_1_light'])
    rl2 = float(row['right_eye_2_light'])
    rl3 = float(row['right_eye_3_light'])

    feature_map[base_id] = [age, days, rl1, rl2, rl3]  # 5 features total

# =========================
# DATASET / DATALOADER
# =========================
class TriRightEyeDatasetWithFeatures(Dataset):
    def __init__(self, data, transform, feature_map):
        self.data = data
        self.transform = transform
        self.feature_map = feature_map

    def __len__(self):
        return len(self.data['label'])

    def __getitem__(self, idx):
        images = [self.data[k][idx] for k in ['r1','r2','r3']]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        base_id = self.data['base_ids'][idx]
        features = self.feature_map.get(base_id, [0.0]*5)
        return images, label, torch.tensor(features, dtype=torch.float32)

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type == 'cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
        persistent_workers=False
    )

# =========================
# MODEL (3x ResNet18 + MLP head + Feature Fusion)
# =========================
class TriRightResNetWithFeatures(nn.Module):
    def __init__(self):
        super().__init__()
        def res():
            m = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
            m.fc = nn.Identity()
            return m
        self.models = nn.ModuleList([res() for _ in range(3)])  # Only 3 models
        self.fc = nn.Sequential(
            nn.Linear(3*512, 1024),
            nn.LayerNorm(1024),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128)
        )
        # 5 features: age, days, rl1, rl2, rl3
        self.feature_fc = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        self.final_fc = nn.Sequential(
            nn.Linear(128 + 32, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, *x, features=None):
        feats = [model(xi) for model, xi in zip(self.models, x)]
        x_img = torch.cat(feats, dim=1)
        x_img = self.fc(x_img)
        x_feat = self.feature_fc(features)
        x = torch.cat([x_img, x_feat], dim=1)
        return self.final_fc(x)

# =========================
# EVALUATION
# =========================
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    for imgs, labels, features in loader:
        imgs = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        features = features.to(device).float()
        out = model(*imgs, features=features)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(all_labels)) < 2:
        p = r = f1 = auc = float('nan')
        acc = accuracy_score(all_labels, all_preds)
        tn = fp = fn = tp = 0
        try:
            tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
        except Exception:
            pass
        return p, r, f1, acc, auc, tp, tn, fp, fn

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn

# =========================
# PREPARE DATASETS
# =========================
train_data = prepare_dataset_strict(train_dirs_anemic, train_dirs_non, "TRAIN")
val_data = prepare_dataset_strict(val_dirs_anemic, val_dirs_non, "VAL")
test_data = prepare_dataset_strict(test_dirs_anemic, test_dirs_non, "TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No tri-matched TRAIN samples found. Check paths/filenames.")
if len(val_data['label']) == 0:
    print("⚠️ No tri-matched VAL samples found.")
if len(test_data['label']) == 0:
    print("⚠️ No tri-matched TEST samples found.")

# =========================
# 5-FOLD CV
# =========================
results = []

for resolution in RESOLUTIONS:
    print(f"\n===== Processing resolution: {resolution} =====")
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    if len(train_data['label']) < N_SPLITS:
        raise RuntimeError(f"Too few train samples for {N_SPLITS}-fold CV.")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(train_data['label'])
    fold = 1
    cv_index_records = []

    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- Fold {fold} ---")

        cv_index_records.append({
            "fold": fold,
            "train_indices": train_idx.tolist(),
            "val_indices": val_idx.tolist()
        })

        train_subset = {k: [v[i] for i in train_idx] for k, v in train_data.items()}
        val_subset = {k: [v[i] for i in val_idx] for k, v in train_data.items()}

        train_loader = make_loader(TriRightEyeDatasetWithFeatures(train_subset, train_tf, feature_map), batch_size=BATCH_CV, shuffle=True)
        val_loader = make_loader(TriRightEyeDatasetWithFeatures(val_subset, eval_tf, feature_map), batch_size=BATCH_CV, shuffle=False)

        model = TriRightResNetWithFeatures().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels, features in train_loader:
                imgs = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                features = features.to(device).float()
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(*imgs, features=features)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()
            print(f"Epoch [{epoch+1}/{EPOCHS_CV}] 🔁 Loss: {total_loss:.6f}")

        val_metrics = evaluate(model, val_loader)
        row = {
            'Resolution': resolution,
            'Fold': fold,
            'Epochs': EPOCHS_CV,
            'BatchSize': BATCH_CV,
            'LearningRate': LR_CV,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8]
        }
        results.append(row)
        print(row)

        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"cv_fold_{fold}_res{resolution}.pt")
            torch.save({
                'model_state': model.state_dict(),
                'seed': SEED,
                'resolution': resolution
            }, fold_path)

        fold += 1

    # Save results
    if results:
        pd.DataFrame(results).to_csv(os.path.join(output_dir, f"{resolution}_val_cross_validation_results_tri_right.csv"), index=False)
    pd.DataFrame(cv_index_records).to_json(os.path.join(output_dir, f"{resolution}_cv_indices.json"), orient='records', indent=2)

    # Select best fold
    results_df = pd.DataFrame(results)
    results_df['min_PR'] = results_df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = results_df[(results_df['Val_Precision'] >= 0.90) & (results_df['Val_Recall'] >= 0.90)]
    best = candidates.sort_values(['Val_F1','Val_AUC','min_PR'], ascending=False).iloc[0] if len(candidates) > 0 else \
            results_df.sort_values(['min_PR','Val_F1','Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # Load best fold model
    best_model = TriRightResNetWithFeatures().to(device)
    best_model_path = os.path.join(output_dir, f"cv_fold_{best_fold}_res{resolution}.pt")
    state = torch.load(best_model_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Evaluate on TEST using best fold
    if len(test_data['label']) > 0:
        test_loader = make_loader(TriRightEyeDatasetWithFeatures(test_data, eval_tf, feature_map), batch_size=BATCH_CV, shuffle=False)
        test_metrics = evaluate(best_model, test_loader)
        print("\n📊 TEST (best-fold model) Results")
        print(f"Precision: {test_metrics[0]:.4f}, Recall: {test_metrics[1]:.4f}, F1: {test_metrics[2]:.4f}")
        print(f"Accuracy:  {test_metrics[3]:.4f}, AUC: {test_metrics[4]:.4f}")

        pd.DataFrame([{
            'ChosenFold': best_fold,
            'Test_Precision': test_metrics[0], 'Test_Recall': test_metrics[1], 'Test_F1': test_metrics[2],
            'Test_Accuracy': test_metrics[3], 'Test_AUC': test_metrics[4], 'Test_TP': test_metrics[5],
            'Test_TN': test_metrics[6], 'Test_FP': test_metrics[7], 'Test_FN': test_metrics[8]
        }]).to_csv(os.path.join(output_dir, f"{resolution}_bestfold_test_results_tri_right.csv"), index=False)

print("\n✅ Tri-Right-Eye: Done. 5-fold CV, best-fold selection, and TEST evaluation are fully reproducible.")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

🔎 TRAIN - discovered bases: anemic=248, non-anemic=1907
✅ TRAIN: tri-matched samples -> anemic=248, non-anemic=1907, total=2155

🔎 VAL - discovered bases: anemic=37, non-anemic=298
✅ VAL: tri-matched samples -> anemic=37, non-anemic=298, total=335

🔎 TEST - discovered bases: anemic=34, non-anemic=284
✅ TEST: tri-matched samples -> anemic=34, non-anemic=284, total=318

===== Processing resolution: 224 =====

--- Fold 1 ---
Epoch [1/120] 🔁 Loss: 30.467892
Epoch [2/120] 🔁 Loss: 26.470141
Epoch [3/120] 🔁 Loss: 26.170041
Epoch [4/120] 🔁 Loss: 25.320405
Epoch [5/120] 🔁 Loss: 25.367928
Epoch [6/120] 🔁 Loss: 25.166305
Epoch [7/120] 🔁 Loss: 25.139764
Epoch [8/120] 🔁 Loss: 25.038733
Epoch [9/120] 🔁 Loss: 24.174090
Epoch [10/120] 🔁 Loss: 25.039787
Epoch [11/120] 🔁 Loss: 24.684445
Epoch [12/120] 🔁 Loss: 24.603830
Epoch [13/120] 🔁 Loss: 23.980597
Epoch [14/120] 🔁 Loss: 24.086990
Epoch [15/120] 🔁 Loss: 23.292998
Epoch [16/120] 🔁 Loss: 23.676232
Epoch [1

In [ ]:
import os
import random
import numpy as np
import cv2
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet18, ResNet18_Weights
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, roc_auc_score, confusion_matrix
from torch.cuda.amp import GradScaler, autocast
from sklearn.model_selection import StratifiedKFold
import warnings
warnings.filterwarnings("ignore")

# =========================
# GLOBAL CONFIG
# =========================


SEED = 42
NUM_WORKERS = 0          # keep 0 for strict reproducibility
PIN_MEMORY = False       # can be True on CUDA; doesn't affect determinism
USE_AMP = True           # AMP is deterministic under fixed seeds + deterministic algos
SAVE_EVERY_FOLD_MODEL = True
N_SPLITS = 5
RESOLUTIONS = [224]
EPOCHS_CV = 60
BATCH_CV = 18
LR_CV = 0.00017

# =========================
# DEVICE + DETERMINISM
# =========================
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    try:
        torch.use_deterministic_algorithms(True, warn_only=True)
    except Exception:
        pass
    try:
        torch.backends.cuda.matmul.allow_tf32 = False
        torch.backends.cudnn.allow_tf32 = False
    except Exception:
        pass

try:
    cv2.setNumThreads(0)
except Exception:
    pass

set_global_seed(SEED)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))

# =========================
# PATHS
# =========================
base_path = "/home/ubuntu/anemia-storage/hb_mobilenet/mat_conjunctiva_all_consistent_deletion/"
output_dir = os.path.join(base_path, "tri-left-eye_hb_90_repro_bestfold_only_with_age_days")
os.makedirs(output_dir, exist_ok=True)

def make_full_path(subdirs):
    return {k: os.path.join(base_path, v) for k, v in subdirs.items()}

# Define directories (left EYE ONLY)
train_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_train_roi/'
})
train_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_train_roi/'
})
val_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_val_roi/'
})
val_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_val_roi/'
})
test_dirs_anemic = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_test_roi/'
})
test_dirs_non = make_full_path({
    'left1': 'tri_left_eye/left_eye_1_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left2': 'tri_left_eye/left_eye_2_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/',
    'left3': 'tri_left_eye/left_eye_3_hb_less_than_9_0/conjunctiva_extracted/anemic_not_test_roi/'
})

# =========================
# UTILS: base id handling and tri-eye intersections
# =========================
def base_from(fname, suffix):
    if not fname.endswith(suffix):
        return None
    return fname[: -len(suffix)]

def common_bases_across_three(dirs_map):
    suffixes = {
        'left1': '_left_eye_1.png',
        'left2': '_left_eye_2.png',
        'left3': '_left_eye_3.png',
    }
    bases_sets = []
    for k in ['left1','left2','left3']:
        folder = dirs_map[k]
        if not os.path.isdir(folder):
            print(f"⚠️ Folder missing: {folder}")
            return []
        names = sorted([f for f in os.listdir(folder) if f.endswith(suffixes[k])])
        bases = set()
        for f in names:
            b = base_from(f, suffixes[k])
            if b:
                bases.add(b)
        bases_sets.append(bases)
    common = set.intersection(*bases_sets) if bases_sets else set()
    return sorted(list(common))

def load_tri_images_by_bases(dirs_map, bases):
    out = {'r1': [], 'r2': [], 'r3': []}
    key_map = {
        'r1': ('left1', '_left_eye_1.png'),
        'r2': ('left2', '_left_eye_2.png'),
        'r3': ('left3', '_left_eye_3.png'),
    }
    ok = 0
    for b in bases:
        imgs = {}
        failed = False
        for short_k, (long_k, suf) in key_map.items():
            path = os.path.join(dirs_map[long_k], b + suf)
            img = cv2.imread(path)
            if img is None:
                failed = True
                break
            imgs[short_k] = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if failed:
            continue
        for k in out.keys():
            out[k].append(imgs[k])
        ok += 1
    return out, ok

def prepare_dataset_strict(anemic_dirs, non_dirs, split_name="(split)"):
    bases_anemic = common_bases_across_three(anemic_dirs)
    bases_non = common_bases_across_three(non_dirs)
    print(f"\n🔎 {split_name} - discovered bases: anemic={len(bases_anemic)}, non-anemic={len(bases_non)}")

    imgs_a, _ = load_tri_images_by_bases(anemic_dirs, bases_anemic)
    imgs_n, _ = load_tri_images_by_bases(non_dirs, bases_non)

    data = {
        'r1': imgs_a['r1'] + imgs_n['r1'],
        'r2': imgs_a['r2'] + imgs_n['r2'],
        'r3': imgs_a['r3'] + imgs_n['r3'],
        'label': [1]*len(imgs_a['r1']) + [0]*len(imgs_n['r1']),
        'base_ids': bases_anemic + bases_non  # ✅ Store base IDs
    }

    print(f"✅ {split_name}: tri-matched samples -> anemic={len(imgs_a['r1'])}, non-anemic={len(imgs_n['r1'])}, total={len(data['label'])}")
    return data

# =========================
# LOAD DATAFRAME WITH AGE, DAYS, AND LIGHTING
# =========================
df = pd.read_csv("/home/ubuntu/anemia-storage/hb_mobilenet/dataset_imputed.csv")

# ✅ Extract base_id from filename
def extract_base_id(filename):
    for suffix in ['_left_eye_1.png', '_left_eye_2.png', '_left_eye_3.png']:
        if filename.endswith(suffix):
            return filename[:-len(suffix)]
    return filename

# ✅ Create feature_map using clean base_id
feature_map = {}
for _, row in df.iterrows():
    raw_id = row['left_eye_1']
    base_id = extract_base_id(raw_id)
    age = float(row['age_at_registration_final'])
    days = float(row['days_since_lmp_final'])
    rl1 = float(row['left_eye_1_light'])
    rl2 = float(row['left_eye_2_light'])
    rl3 = float(row['left_eye_3_light'])

    feature_map[base_id] = [age, days, rl1, rl2, rl3]  # 5 features total

# =========================
# DATASET / DATALOADER
# =========================
class TrileftEyeDatasetWithFeatures(Dataset):
    def __init__(self, data, transform, feature_map):
        self.data = data
        self.transform = transform
        self.feature_map = feature_map

    def __len__(self):
        return len(self.data['label'])

    def __getitem__(self, idx):
        images = [self.data[k][idx] for k in ['r1','r2','r3']]
        images = [self.transform(img) for img in images]
        label = self.data['label'][idx]
        base_id = self.data['base_ids'][idx]
        features = self.feature_map.get(base_id, [0.0]*5)
        return images, label, torch.tensor(features, dtype=torch.float32)

def seed_worker(worker_id):
    worker_seed = SEED + worker_id
    np.random.seed(worker_seed)
    random.seed(worker_seed)
    torch.manual_seed(worker_seed)

def make_loader(dataset, batch_size, shuffle):
    g = torch.Generator()
    g.manual_seed(SEED)
    return DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY and (device.type == 'cuda'),
        worker_init_fn=seed_worker if NUM_WORKERS > 0 else None,
        generator=g,
        persistent_workers=False
    )

# =========================
# MODEL (3x ResNet18 + MLP head + Feature Fusion)
# =========================
class TrileftResNetWithFeatures(nn.Module):
    def __init__(self):
        super().__init__()
        def res():
            m = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
            m.fc = nn.Identity()
            return m
        self.models = nn.ModuleList([res() for _ in range(3)])  # Only 3 models
        self.fc = nn.Sequential(
            nn.Linear(3*512, 1024),
            nn.LayerNorm(1024),
            nn.GELU(),
            nn.Dropout(0.5),
            nn.Linear(1024, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128)
        )
        # 5 features: age, days, rl1, rl2, rl3
        self.feature_fc = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        self.final_fc = nn.Sequential(
            nn.Linear(128 + 32, 64),
            nn.LayerNorm(64),
            nn.GELU(),
            nn.Dropout(0.3),
            nn.Linear(64, 1)
        )

    def forward(self, *x, features=None):
        feats = [model(xi) for model, xi in zip(self.models, x)]
        x_img = torch.cat(feats, dim=1)
        x_img = self.fc(x_img)
        x_feat = self.feature_fc(features)
        x = torch.cat([x_img, x_feat], dim=1)
        return self.final_fc(x)

# =========================
# EVALUATION
# =========================
@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []
    for imgs, labels, features in loader:
        imgs = [img.to(device).float() for img in imgs]
        labels = labels.to(device).float().unsqueeze(1)
        features = features.to(device).float()
        out = model(*imgs, features=features)
        prob = torch.sigmoid(out).cpu().numpy().flatten()
        pred = (prob > 0.5).astype(int)
        all_preds.extend(pred.tolist())
        all_probs.extend(prob.tolist())
        all_labels.extend(labels.cpu().numpy().flatten().tolist())

    if len(set(all_labels)) < 2:
        p = r = f1 = auc = float('nan')
        acc = accuracy_score(all_labels, all_preds)
        tn = fp = fn = tp = 0
        try:
            tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
        except Exception:
            pass
        return p, r, f1, acc, auc, tp, tn, fp, fn

    p, r, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='binary')
    acc = accuracy_score(all_labels, all_preds)
    auc = roc_auc_score(all_labels, all_probs)
    tn, fp, fn, tp = confusion_matrix(all_labels, all_preds, labels=[0,1]).ravel()
    return p, r, f1, acc, auc, tp, tn, fp, fn

# =========================
# PREPARE DATASETS
# =========================
train_data = prepare_dataset_strict(train_dirs_anemic, train_dirs_non, "TRAIN")
val_data = prepare_dataset_strict(val_dirs_anemic, val_dirs_non, "VAL")
test_data = prepare_dataset_strict(test_dirs_anemic, test_dirs_non, "TEST")

if len(train_data['label']) == 0:
    raise RuntimeError("No tri-matched TRAIN samples found. Check paths/filenames.")
if len(val_data['label']) == 0:
    print("⚠️ No tri-matched VAL samples found.")
if len(test_data['label']) == 0:
    print("⚠️ No tri-matched TEST samples found.")

# =========================
# 5-FOLD CV
# =========================
results = []

for resolution in RESOLUTIONS:
    print(f"\n===== Processing resolution: {resolution} =====")
    train_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomVerticalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    eval_tf = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((resolution, resolution)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    if len(train_data['label']) < N_SPLITS:
        raise RuntimeError(f"Too few train samples for {N_SPLITS}-fold CV.")

    kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=SEED)
    labels_np = np.array(train_data['label'])
    fold = 1
    cv_index_records = []

    for train_idx, val_idx in kf.split(np.zeros_like(labels_np), labels_np):
        print(f"\n--- Fold {fold} ---")

        cv_index_records.append({
            "fold": fold,
            "train_indices": train_idx.tolist(),
            "val_indices": val_idx.tolist()
        })

        train_subset = {k: [v[i] for i in train_idx] for k, v in train_data.items()}
        val_subset = {k: [v[i] for i in val_idx] for k, v in train_data.items()}

        train_loader = make_loader(TrileftEyeDatasetWithFeatures(train_subset, train_tf, feature_map), batch_size=BATCH_CV, shuffle=True)
        val_loader = make_loader(TrileftEyeDatasetWithFeatures(val_subset, eval_tf, feature_map), batch_size=BATCH_CV, shuffle=False)

        model = TrileftResNetWithFeatures().to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = optim.Adam(model.parameters(), lr=LR_CV)
        scaler = GradScaler(enabled=(USE_AMP and device.type == "cuda"))

        for epoch in range(EPOCHS_CV):
            model.train()
            total_loss = 0.0
            for imgs, labels, features in train_loader:
                imgs = [img.to(device).float() for img in imgs]
                labels = labels.to(device).float().unsqueeze(1)
                features = features.to(device).float()
                optimizer.zero_grad(set_to_none=True)
                with autocast(enabled=(USE_AMP and device.type == "cuda")):
                    out = model(*imgs, features=features)
                    loss = criterion(out, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
                total_loss += loss.item()
            print(f"Epoch [{epoch+1}/{EPOCHS_CV}] 🔁 Loss: {total_loss:.6f}")

        val_metrics = evaluate(model, val_loader)
        row = {
            'Resolution': resolution,
            'Fold': fold,
            'Epochs': EPOCHS_CV,
            'BatchSize': BATCH_CV,
            'LearningRate': LR_CV,
            'Val_Precision': val_metrics[0],
            'Val_Recall': val_metrics[1],
            'Val_F1': val_metrics[2],
            'Val_Accuracy': val_metrics[3],
            'Val_AUC': val_metrics[4],
            'Val_TP': val_metrics[5],
            'Val_TN': val_metrics[6],
            'Val_FP': val_metrics[7],
            'Val_FN': val_metrics[8]
        }
        results.append(row)
        print(row)

        if SAVE_EVERY_FOLD_MODEL:
            fold_path = os.path.join(output_dir, f"cv_fold_{fold}_res{resolution}.pt")
            torch.save({
                'model_state': model.state_dict(),
                'seed': SEED,
                'resolution': resolution
            }, fold_path)

        fold += 1

    # Save results
    if results:
        pd.DataFrame(results).to_csv(os.path.join(output_dir, f"{resolution}_val_cross_validation_results_tri_left.csv"), index=False)
    pd.DataFrame(cv_index_records).to_json(os.path.join(output_dir, f"{resolution}_cv_indices.json"), orient='records', indent=2)

    # Select best fold
    results_df = pd.DataFrame(results)
    results_df['min_PR'] = results_df[['Val_Precision','Val_Recall']].min(axis=1)
    candidates = results_df[(results_df['Val_Precision'] >= 0.90) & (results_df['Val_Recall'] >= 0.90)]
    best = candidates.sort_values(['Val_F1','Val_AUC','min_PR'], ascending=False).iloc[0] if len(candidates) > 0 else \
            results_df.sort_values(['min_PR','Val_F1','Val_AUC'], ascending=False).iloc[0]

    best_fold = int(best['Fold'])
    print(f"✅ Best fold = {best_fold}")

    # Load best fold model
    best_model = TrileftResNetWithFeatures().to(device)
    best_model_path = os.path.join(output_dir, f"cv_fold_{best_fold}_res{resolution}.pt")
    state = torch.load(best_model_path, map_location=device)
    best_model.load_state_dict(state['model_state'])

    # Evaluate on TEST using best fold
    if len(test_data['label']) > 0:
        test_loader = make_loader(TrileftEyeDatasetWithFeatures(test_data, eval_tf, feature_map), batch_size=BATCH_CV, shuffle=False)
        test_metrics = evaluate(best_model, test_loader)
        print("\n📊 TEST (best-fold model) Results")
        print(f"Precision: {test_metrics[0]:.4f}, Recall: {test_metrics[1]:.4f}, F1: {test_metrics[2]:.4f}")
        print(f"Accuracy:  {test_metrics[3]:.4f}, AUC: {test_metrics[4]:.4f}")

        pd.DataFrame([{
            'ChosenFold': best_fold,
            'Test_Precision': test_metrics[0], 'Test_Recall': test_metrics[1], 'Test_F1': test_metrics[2],
            'Test_Accuracy': test_metrics[3], 'Test_AUC': test_metrics[4], 'Test_TP': test_metrics[5],
            'Test_TN': test_metrics[6], 'Test_FP': test_metrics[7], 'Test_FN': test_metrics[8]
        }]).to_csv(os.path.join(output_dir, f"{resolution}_bestfold_test_results_tri_left.csv"), index=False)

print("\n✅ Tri-left-Eye: Done. 5-fold CV, best-fold selection, and TEST evaluation are fully reproducible.")

Using device: cuda
GPU: NVIDIA H100 80GB HBM3

🔎 TRAIN - discovered bases: anemic=243, non-anemic=1857
✅ TRAIN: tri-matched samples -> anemic=243, non-anemic=1857, total=2100

🔎 VAL - discovered bases: anemic=39, non-anemic=294
✅ VAL: tri-matched samples -> anemic=39, non-anemic=294, total=333

🔎 TEST - discovered bases: anemic=35, non-anemic=270
✅ TEST: tri-matched samples -> anemic=35, non-anemic=270, total=305

===== Processing resolution: 224 =====

--- Fold 1 ---
Epoch [1/60] 🔁 Loss: 40.116294
Epoch [2/60] 🔁 Loss: 34.522666
Epoch [3/60] 🔁 Loss: 34.022404
Epoch [4/60] 🔁 Loss: 33.561304
Epoch [5/60] 🔁 Loss: 32.779696
Epoch [6/60] 🔁 Loss: 31.862547
Epoch [7/60] 🔁 Loss: 31.765271
Epoch [8/60] 🔁 Loss: 30.737337
Epoch [9/60] 🔁 Loss: 30.733905
Epoch [10/60] 🔁 Loss: 29.635578
Epoch [11/60] 🔁 Loss: 29.762556
Epoch [12/60] 🔁 Loss: 29.926157
Epoch [13/60] 🔁 Loss: 29.133246
Epoch [14/60] 🔁 Loss: 27.053476
Epoch [15/60] 🔁 Loss: 27.675854
Epoch [16/60] 🔁 Loss: 26.900820
Epoch [17/60] 🔁 Loss: 25